In [ ]:
from fastai.vision.all import *
import librosa as librosa

In [ ]:
meta = pd.read_csv('../input/birdclef-2022/train_metadata.csv')
meta.head(3)

In [ ]:
eBird = pd.read_csv('../input/birdclef-2022/eBird_Taxonomy_v2021.csv')
eBird.head(2)

**CARGANDO AUDIOS Y CREANDO ESPECTROGRAMAS**

In [ ]:
# Copiado de https://www.kaggle.com/code/johnowhitaker/baseline-model/notebook
# Fuentes: 
#    https://medium.com/analytics-vidhya/understanding-the-mel-spectrogram-fca2afa2ce53
#    https://stackoverflow.com/questions/62584184/understanding-the-shape-of-spectrograms-and-n-mels
#    https://librosa.org/doc/0.7.2/generated/librosa.feature.melspectrogram.html
#    https://es.wikipedia.org/wiki/Escala_Mel
# Explicación:
#   Convertir un trozo de audio de 5s en una imagen que lo representa
#     y = Audio time-series    sr = Sampling rate of y       
#     n_fft = length of the FFT window    (FFT) -> fast Fourier transform ,algoritmo que descompone una onda en un conjunto de ondas seno y coseno
#                                                  Este se aplica a fragmentos de longitud   n_FFT
#     hop_length = Distancia entre los fragmentos. Si  hop_length = n_fft, la cantidad de framentos sería tamaño_tot / n_fft. Sin embargo, normal-
#                                             -mente, los fragmentos se solapan entre ellos. Pudiendo ser quizas la distancia n_fft/2 por ejemplo.
#     n_mels = Numero de separaciones (equidistantes ante la percepción humana), del espectro de frecuencia
#     fmin y fmax = Máximo y Mínmo valor en Hz
#
#     power_to_db = Convert a power spectrogram (amplitude squared) to decibel (dB) units
def chunk_to_spec(chunk, SPEC_HEIGHT=64,SPEC_WIDTH=256, rate=32000, FMIN=200, FMAX=12500):
    mel_spec = librosa.feature.melspectrogram(y=chunk, 
                                              sr=32000, 
                                              n_fft=1024, 
                                              hop_length=int(32000 * 5 / (SPEC_WIDTH - 1)), 
                                              n_mels=SPEC_HEIGHT, 
                                              fmin=FMIN, 
                                              fmax=FMAX)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    return mel_spec_db

In [ ]:
#  Cargamos los Peaks extraidos en el notebook auxiliar
df = pd.read_csv('../input/peak-identification-rubio/info_df.csv')
print(df.shape)
print(df.fn[0])
df.head(2)

In [ ]:
#  Muestra como Cargar una fila del fichero de Picos y cortar el audio correspondiente y el primer pico de la lista
#
#  Fuentes:
#      https://librosa.org/doc/main/generated/librosa.load.html
#  Explicación:
#      fn = file name,    l = length,    y = label(s?),    peaks = picos
#      pandas.sample() = Devuelve una fila aleatoria, o una fila dada, o un porcentaje de filas dado .sample(frac = .25)
#
#      librosa.load(fn = (nombre del archivo de audio), sr=(sound rate), offset=(inicio de fragmento elegido), duration=(5 segundos))
#          -> devueleve:   y = audio time-series,    sr = sound rate
#
#      Tiempo inicial del corte es:
#         - en el caso de que el audio original sea menor de 5s -> ¿negativo?
#                                                                    ¿BUG?
#         - en el caso de que el pico elegido esté mas allá de los últimos 5s del audio original -> los ultimos 5s
#         - si el pico elegido está antes de los ultimos 5s, 2.5s antes del pico comieza la rodaja. Si este inicio de rodaja es menor de 2.5, pues 2.5
#           CREO QUE ESTO ES UN BUG DEL AUTOR YA QUE NO TIENE SENTIDO. CREO QUE QUIERE REFLEJAR QUE SI EL PTO INICAL DE LA RODAJA ES < 0 ENTONCES LA
#           RODAJA COMIENZA EN CERO
# 
#  Finalmente convierte el chunck a spectrograma y lo pinta

fn, l, y, peaks = df.sample().values[0]
#start_time = min(l/32000 - 5, max(float(peaks.split('#')[0])-2.5, 2.5)) # Tiempo inicial del corte
start_time = min(l/32000 - 5, max(float(peaks.split('#')[0])-2.5, 0)) # Correjido 
y, sr = librosa.load(fn, sr=32000, offset=start_time, duration=5)
#plt.imshow(chunk_to_spec(y, SPEC_HEIGHT=128), cmap='inferno')
prueba = chunk_to_spec(y, SPEC_HEIGHT=128)
prueba

In [ ]:
tensor(prueba)

**IMPLEMENTACION CON LBRERIA FAST-AI**

In [ ]:
# Fuente general: 
#     https://docs.fast.ai/tutorial.vision.html#Classifying-breeds
# Otras fuentes:
#     https://fastcore.fast.ai/basics.html#fastuple
#     https://www.udacity.com/blog/2021/11/__init__-in-python-an-overview.html#:~:text=The%20__init__%20method%20is%20the%20Python%20equivalent%20of,is%20only%20used%20within%20classes.
#     https://fastcore.fast.ai/basics.html#uniqueify
#     pandas.iloc - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html
#     torch.unsqueeze - https://stackoverflow.com/questions/57237352/what-does-unsqueeze-do-in-pytorch
#     torch.cat - https://pytorch.org/docs/stable/generated/torch.cat.html
#     https://fastcore.fast.ai/transform#Transform
#     
#     
#     
# Explicación:
#   
#   TitledImage = ???     ¿fastuple? -> "tupla con funciones extendidas" ( https://fastcore.fast.ai/basics.html#fastuple )
#    
#   ClipTransform = 
#      __init__ = funcion de creación del objeto  
#           uniqueify --> 
#               deja una lista de las clases ordenada en self.vocab
#               guarda clave y par "indice:nombre" -> (0:golondrina, 1:gaviota, 2:aguila)
#
#       encodes = SOLO SE EJECUTA CUANDO SE LE LLAMA.   En este caso solo se ejecuta cuando se llama DENTRO DE LA FUNCION "TFMDLISTS"
#
#            df.iloc[i].values -> coge la fila "i" de "df" extrae sus valores
#            guarda f=nombreFichero,  l=legth,  label=label,  peaks=picos de la fila número "y"
#            elije un número de pico -> ¿solo del 1 al 6?¿Para que elegimos 20?
#            obtiene el tiempo de corte y saca el clip
#            lo transforma a espectrograma -> ¿Por que con power_to_db?
#            
#            CAMBIOS INJUSTIFICADOS AL SPEC ->  ¿spec -= np.min(spec)?   ¿spec /= 80?
#            
#            ¿ torch.unsqueeze(tensor(spec), 0) ?
#               tensor(spec) -> transforma la matriz multidimensional en un tensor 
#               torch.unsqueeze(x, 0) -> unsqueeze significa "descomprimir".  -> añade una dimension al tensor. -> si 0, lo añade en la primera "capa"  
#                      si x = [1, 2, 3] (tensor de dim 2 ~ vector) -> x.unsqueeze(0) = [[1, 2, 3]]   
#                              dim x0 = (3)  dim x' = (1,3) (dentro del 1er [] hay 1[], este con 3 elementos dentro)
#               torch.cat -> Concatena tres veces el mismo tensor "para sortear que las redes usan 3 canales rojo verde y azul" -> ¿¿Muy ineficiente??
#               return spec, self.o2i[label] -> Entiendo que: 
#                         devuelve la entrada "spec" que son los 3 tensores encadenados
#                         Junto con "y", que es el numero de la clase a la ue peertenece. Lo obtiene buscado en la lista clave:valor "self.o2i"
#
#       decodes -> creo que es para tema de impementar funciones de visualizacion ".show" propias de FastAI
#             
#
#
#
         
class TitledImage(fastuple):
    def show(self, ctx=None, **kwargs): show_titled_image(self, ctx=ctx, **kwargs)

class ClipTransform(ItemTransform):

    def __init__(self, df):
        self.df=df
        self.vocab,self.o2i = uniqueify(df['label'], sort=True, bidir=True)
        
    def encodes(self, i, from_np=False):
        f, l, label, peaks = df.iloc[i].values
        clip_num = random.choice([0, 0, 0, 1, 1, 2, 3, 4, 5, 6]) # More prob chose big peak
        start_time = min(l/32000 - 5, max(float(peaks.split('#')[clip_num])-2.5, 0))
        y, sr = librosa.load(f, sr=32000, offset=start_time, duration=5)
        spec = chunk_to_spec(y,SPEC_HEIGHT=112,SPEC_WIDTH=224) # ¿Por que usa esta funcion que lleva power_to_dB y no pcen?
        #¿una vez detectado el pico es mejor pq no adultera tanto la informacion de la que aprendes, al ser mas homogénea?
        spec -= np.min(spec) 
        spec /= 80 # np.max(spec) # Normalize
        spec =  torch.unsqueeze(tensor(spec), 0)
        spec = torch.cat([spec, spec, spec]) # Stack three channels to simulate RGB if using a pretrained model
        return spec, self.o2i[label]
    
    def decodes(self, x):
        return TitledImage(x[0],self.vocab[x[1]])
   

#   Fuentes:
#     https://docs.fast.ai/data.load.html#DataLoader
#     
#     
#   Mas allá de las clases: 
#      ¿se hace una copia de df en df_small?  --> Totalmente innecesario
#      se crea clip_tfm que es un objeto de la clase ClipTransform con las reglas de transformacion de un audio en un solo clip
#      train = 80% random del df
#      Crea índices de train, y de las filas no incluidas entrain pero si en df (que es el indice de validación)
#      
#      Crea las transformaciones de los conjuntos de entrenamiento y validación
#           train_tl= TfmdLists(train_idx, clip_tfm)   ->    le pasa la lista de audios(el índice en df), y el objeto que lo transforma (que ya contiene df)  
#           valid_tl= TfmdLists(valid_idx, clip_tfm)   ->    Hace lo mismo con la validación
#      
#      Crea objeto DataLoader "dls" al que le mete la lista de parejas (imput, label) de training y las de validation
#              train_tl y valid_tl  ->  Son ?? ¿datasets?
#              bs = Batch Size -> nº de samples por que se cogen por cada calculo del gradiente
#              dls = dls.cuda()  ->  para "hacer uso de la gpu"  -> mi gpu no tiene nucleos cuda ¿¿¿???
#
#
 

df_small = df
clip_tfm = ClipTransform(df)
train =  df_small.sample(frac=0.8)
train_idx, valid_idx = list(train.index), df_small[~df_small.index.isin(train.index)].index
print('train and val size', len(train_idx), len(valid_idx))
train_tl= TfmdLists(train_idx, clip_tfm)
valid_tl= TfmdLists(valid_idx, clip_tfm)
dls = DataLoaders.from_dsets(train_tl, valid_tl, bs=16)


dls = dls.cuda()
xb, yb = dls.one_batch()
print(xb.shape)
dls.show_batch(max_n=3)

In [ ]:
#Pruebas con tensores
x = torch.tensor([1, 2, 3, 4])
print(x.shape)
print(x)
print("------")

x = x.unsqueeze(1)
print(x.shape)
print(x)
x = x.squeeze(1)
print(x.shape)
print(x)
x = x.unsqueeze(0)
print(x.shape)
print(x)

In [ ]:
learn = cnn_learner(dls, models.resnet18, loss_func=FocalLossFlat(), metrics=[accuracy], cbs=[ShowGraphCallback(), CSVLogger()])

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-4, 1e-3))

In [ ]:
learn.recorder.plot_loss()
plt.savefig('loss_plot.png')

In [ ]:
learn.save('stage-1')

In [ ]:
learn.remove_cb(CSVLogger) # Not pickleable
learn.export('baseline_3e.pkl')